In [23]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_rows', None)

In [24]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

In [25]:
df_연결현금흐름 = pd.read_csv("./datasets/raw/연결현금흐름.csv",encoding='cp949')
df_개별현금흐름 = pd.read_csv("./datasets/raw/개별현금흐름.csv",encoding='cp949')

In [26]:
# 연결 거래소코드패딩, 회계년도만표시, 컬럼에서 'IFRS연결', ' ', ''제거
df_연결현금흐름["거래소코드"] = df_연결현금흐름["거래소코드"].map(padding) # 거래소코드 padding
df_연결현금흐름["회계년도"] = df_연결현금흐름["회계년도"].map(change_year) # 년도만 표시

list_col = list() # 컬럼명에서 "IFRS연결", " ", "," 제거
for i in list(df_연결현금흐름.columns):
    list_col.append(func_replace_col1(i))
df_연결현금흐름.columns = list_col

df_연결현금흐름.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_연결현금흐름.reset_index(drop=True, inplace=True)

df_연결현금흐름.to_csv("./연결현금흐름1.csv")
df_연결현금흐름.shape

(21017, 6)

In [27]:
# 개별 거래소코드패딩, 회계년도만표시, 컬럼에서 'IFRS연결', ' ', ''제거
df_개별현금흐름["거래소코드"] = df_개별현금흐름["거래소코드"].map(padding) # 거래소코드 padding
df_개별현금흐름["회계년도"] = df_개별현금흐름["회계년도"].map(change_year) # 년도만 표시

list_col = list() # 컬럼명에서 "IFRS연결", " ", "," 제거
for i in list(df_개별현금흐름.columns):
    list_col.append(func_replace_col1(i))
df_개별현금흐름.columns = list_col

df_개별현금흐름.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_개별현금흐름.reset_index(drop=True, inplace=True)

df_개별현금흐름.to_csv("./개별현금흐름1.csv")
df_개별현금흐름.shape

(21017, 6)

In [28]:
# 연결현금흐름 외국계기업 제거

def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

print("제거할 외국계기업 데이터 수 : ", len(df_연결현금흐름[df_연결현금흐름["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
df_연결현금흐름 = df_연결현금흐름[df_연결현금흐름["거래소코드"].map(func_isnt_fcompany)]
df_연결현금흐름.shape # 외국계기업 데이터 204개 제거

제거할 외국계기업 데이터 수 :  204


(20813, 6)

In [29]:
# 개별현금흐름 외국계기업 제거

def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

print("제거할 외국계기업 데이터 수 : ", len(df_개별현금흐름[df_개별현금흐름["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
df_개별현금흐름 = df_개별현금흐름[df_개별현금흐름["거래소코드"].map(func_isnt_fcompany)]
df_개별현금흐름.shape # 외국계기업 데이터 204개 제거

제거할 외국계기업 데이터 수 :  204


(20813, 6)

In [30]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_연결현금흐름["회사명"].unique()))
print("거래소코드수 : ", len(df_연결현금흐름["거래소코드"].unique()))

회사수 :  2490
거래소코드수 :  2518


In [31]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_개별현금흐름["회사명"].unique()))
print("거래소코드수 : ", len(df_개별현금흐름["거래소코드"].unique()))

회사수 :  2490
거래소코드수 :  2518


In [32]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df_연결현금흐름["회사명"].unique()):
    if len(df_연결현금흐름[df_연결현금흐름["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")
# list_overlap_code

거래소코드가 2개인 기업 수 29 개


In [33]:
# 거래소코드 2개인 기업 리스트
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)신성이엔지',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)원익피앤이',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)판타지오',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '삼성물산(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '원텍(주)',
 '줌인터넷(주)',
 '케이엠제약(주)',
 '패션플랫폼(주)']

In [34]:
# 연결 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_연결현금흐름[df_연결현금흐름["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_연결현금흐름["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df_연결현금흐름[df_연결현금흐름["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df_연결현금흐름 = pd.concat(dfs)
df_연결현금흐름.reset_index(drop=True, inplace=True)

In [35]:
# 개별 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_개별현금흐름[df_개별현금흐름["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_개별현금흐름["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df_개별현금흐름[df_개별현금흐름["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df_개별현금흐름 = pd.concat(dfs)
df_개별현금흐름.reset_index(drop=True, inplace=True)

In [36]:
# 회사명이 2개인 거래소코드 존재 확인
list_overlap_code = list()
for i in list(df_연결현금흐름["거래소코드"].unique()):
    if len(df_연결현금흐름[df_연결현금흐름["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

['036420']

In [37]:
## 기업명 변경
df_연결현금흐름[df_연결현금흐름['거래소코드']=='036420']
df_연결현금흐름.loc[df_연결현금흐름['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [38]:
## 기업명 변경
df_개별현금흐름[df_개별현금흐름['거래소코드']=='036420']
df_개별현금흐름.loc[df_개별현금흐름['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [39]:
df_연결현금흐름.isna().sum()

회사명                         0
거래소코드                       0
회계년도                        0
영업활동으로인한현금흐름(간접법)(천원)    4750
투자활동으로인한현금흐름(천원)         4750
재무활동으로인한현금흐름(천원)         4750
dtype: int64

In [40]:
df_개별현금흐름.isna().sum()

회사명                        0
거래소코드                      0
회계년도                       0
영업활동으로인한현금흐름(간접법)(천원)    379
투자활동으로인한현금흐름(천원)         379
재무활동으로인한현금흐름(천원)         379
dtype: int64

In [44]:
# df_연결현금흐름 완전데이터와 결측데이터로 나눈 뒤 결측데이터를 개별로 대체시키고, 다시 통합하면 끝!
df_현금흐름_완전 =df_연결현금흐름[df_연결현금흐름["재무활동으로인한현금흐름(천원)"].isna().map(func_not)]

df_현금흐름_결측 =df_연결현금흐름[df_연결현금흐름["재무활동으로인한현금흐름(천원)"].isna()]


In [51]:
# 행전체가 결측치인 것 중 개별재무 데이터로 대체한 컬럼 말고 나머지 결측치 컬럼 리스트(58개))
list_na_col = ['영업활동으로인한현금흐름(간접법)(천원)','투자활동으로인한현금흐름(천원)','재무활동으로인한현금흐름(천원)']


df_현금흐름_결측 = df_현금흐름_결측.drop(list_na_col, axis=1)
df_현금흐름_결측 = pd.merge(df_현금흐름_결측, df_개별현금흐름[["거래소코드","회계년도"]+list_na_col], on=["거래소코드","회계년도"], how = 'left')
# df_현금흐름_결측

df_현금흐름 = pd.concat([df_현금흐름_완전,df_현금흐름_결측], axis=0)

In [52]:
df_현금흐름.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '영업활동으로인한현금흐름(간접법)(천원)': 379,
 '투자활동으로인한현금흐름(천원)': 379,
 '재무활동으로인한현금흐름(천원)': 379}

In [54]:
print("제거행 갯수 : ", len(df_현금흐름[df_현금흐름["영업활동으로인한현금흐름(간접법)(천원)"].isna()]))
print("제거기업 갯수 : ", len(list(df_현금흐름[df_현금흐름["영업활동으로인한현금흐름(간접법)(천원)"].isna()]["거래소코드"].unique())))
# 약 개 기업의 어떤 년도에는 재무,별도 데이터 둘 다 없는 컬럼이 존재하기에 해당 연도 데이터 제거!(1948행)
# 데이터 누락된 사유 : "필수 공시 지표가 아닌 지표는 누락되어 데이터 존재x" 또는 "비상장기업(이었던 기업) 

제거행 갯수 :  379
제거기업 갯수 :  177


In [55]:
df_현금흐름 = df_현금흐름[df_현금흐름["영업활동으로인한현금흐름(간접법)(천원)"].isna().map(func_not)] # 위 결측치 없는 데이터로 필터링
df_현금흐름

df_현금흐름.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '영업활동으로인한현금흐름(간접법)(천원)': 0,
 '투자활동으로인한현금흐름(천원)': 0,
 '재무활동으로인한현금흐름(천원)': 0}

In [56]:
df_현금흐름.shape

(20326, 6)

In [57]:
df_현금흐름.to_csv('./현금흐름.csv', encoding= 'UTF-8-sig')